# Grid Search On LSTM model
This purpose of this notebook is to apply gridsearchCV on an LSTM model which has previously been decided to be the best for this dataset. This notebook does not include data visualizations or expoloartion, only the gridsearch.

In [ ]:
# importing basic packages

import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import plotly.express as px
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go



In [1]:
#mounting the notebook to my drive
from google.colab import drive
drive.mount('/content/drive')
#setting the directory to the proper folder
import os
os.chdir("/content/drive/My Drive/Stock Algorithim/Datasets")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#the start and end date
start_date = dt.datetime(2023,9,15)
end_date = dt.datetime(2023,10,12)

#loading from yahoo finance
spyData = yf.download("SPY",start_date, end_date)

#data.head(-2)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
#spyData=pd.read_csv("spydata.csv",index_col=0)

In [ ]:
spyData.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-10-05,424.359985,425.369995,421.170013,424.500000,424.500000,70142700
2023-10-06,421.970001,431.130005,420.600006,429.540009,429.540009,113202700
2023-10-09,427.579987,432.880005,427.010010,432.290009,432.290009,80374400
2023-10-10,432.940002,437.220001,432.529999,434.540009,434.540009,78607300
2023-10-11,435.640015,436.579987,433.179993,436.320007,436.320007,62451700


In [ ]:
lookback =5

Spliting the data into train and test with an 80:20 ratio without shuffling because time series data must be continuous.

In [ ]:
# Setting 80 percent data for training
training_data_len = math.ceil(len(spyData) * .8)
training_data_len

#Splitting the dataset
train_data = spyData[:training_data_len].iloc[:,3:4]
test_data = spyData[training_data_len:].iloc[:,3:4]
print(train_data.shape, test_data.shape)

(16, 1) (3, 1)


In [ ]:
# Selecting only the Close Price values
dataset_train = train_data.Close.values
# Reshaping 1D to 2D array
dataset_train = np.reshape(dataset_train, (-1,1))
dataset_train.shape

(16, 1)

In [ ]:
# Selecting Close Price values
dataset_test = test_data.Close.values
# Reshaping 1D to 2D array
dataset_test = np.reshape(dataset_test, (-1,1))
# Normalizing values between 0 and 1
dataset_test.shape

(3, 1)

Scaling the data between 0 and 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.fit_transform(dataset_test)

#scaled_train = dataset_train
#scaled_test = dataset_test




Setting up a lookback for the Training and Test Data of the past 50 values

In [ ]:
X_train = []
y_train = []
for i in range(lookback, len(scaled_train)):
	X_train.append(scaled_train[i-lookback:i, 0])
	y_train.append(scaled_train[i, 0])
	print("X",X_train[i-lookback])
	print("Y::",y_train[i-lookback])


X [0.98820283 1.         0.95825712 0.773594   0.44464676]
Y:: 0.4006358287859264
X [1.         0.95825712 0.773594   0.44464676 0.40063583]
Y:: 0.48275909815329854
X [0.95825712 0.773594   0.44464676 0.40063583 0.4827591 ]
Y:: 0.19464641023972362
X [0.773594   0.44464676 0.40063583 0.4827591  0.19464641]
Y:: 0.20235888048717143
X [0.44464676 0.40063583 0.4827591  0.19464641 0.20235888]
Y:: 0.3144278579410056
X [0.40063583 0.4827591  0.19464641 0.20235888 0.31442786]
Y:: 0.26724194033012694
X [0.4827591  0.19464641 0.20235888 0.31442786 0.26724194]
Y:: 0.2595280854381059
X [0.19464641 0.20235888 0.31442786 0.26724194 0.25952809]
Y:: 0.0
X [0.20235888 0.31442786 0.26724194 0.25952809 0.        ]
Y:: 0.13929247431830305
X [0.31442786 0.26724194 0.25952809 0.         0.13929247]
Y:: 0.13203278284480646
X [0.26724194 0.25952809 0.         0.13929247 0.13203278]
Y:: 0.36070821800397823


In [ ]:
X_test = []
y_test = []
for i in range(lookback, len(scaled_test)):
	X_test.append(scaled_test[i-lookback:i, 0])
	y_test.append(scaled_test[i, 0])


The data then needs to be converted to a numpy array and reshaped

In [ ]:
# The data is converted to Numpy array
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)


X_train : (11, 5, 1) y_train : (11, 1)


In [ ]:
# The data is converted to numpy array
X_test, y_test = np.array(X_test), np.array(y_test)

#Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)


IndexError: ignored

The data is now ready to be used by the keras LSTM model.

In [ ]:
!pip install scikeras

In [ ]:
#imports neccesary for the model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, RMSprop, SGD
from keras.initializers import glorot_uniform, he_normal
from keras.callbacks import EarlyStopping

Create a method to define the model based on given paramaters

In [ ]:
def create_model(neurons, optimizer, num_layers, activation, dropout_rate,learning_rate):
    # Define the model as sequential
    model = Sequential()

    # Adding the input layer
    model.add(LSTM(neurons,
					return_sequences = True,
					input_shape = (X_train.shape[1],1)))
    # Loop through the provided number of layers and add layers
    for _ in range(num_layers - 2):  # num_layers - 4 because of the input layer and final
        model.add(LSTM(neurons, activation=activation, return_sequences=True))
       # model.add(Dropout(dropout_rate))

    # The last LSTM layer (no return_sequences=True)
    model.add(LSTM(neurons, activation=activation, return_sequences=False))
  #  model.add(Dropout(dropout_rate))

    # Add a Dense layer with appropriate units and activation
    model.add(Dense(neurons/2))

    model.add(Dense(1))

    # Define optimizer with custom learning rate if needed
    if optimizer == 'Adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'RMSprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        optimizer = SGD(learning_rate=learning_rate)


    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [ ]:
model = create_model(60,'SGD',5,'relu','0.2',0.01)

In [ ]:
model.fit(X_train,
				y_train,
				batch_size = 1,
				epochs = 12)

Epoch 1/12
11/11 [==============================] - 4s 8ms/step - loss: 0.0490 - accuracy: 0.0909
Epoch 2/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0229 - accuracy: 0.0909
Epoch 3/12
11/11 [==============================] - 0s 7ms/step - loss: 0.0192 - accuracy: 0.0909
Epoch 4/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0184 - accuracy: 0.0909
Epoch 5/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0177 - accuracy: 0.0909
Epoch 6/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0183 - accuracy: 0.0909
Epoch 7/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0185 - accuracy: 0.0909
Epoch 8/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0181 - accuracy: 0.0909    
Epoch 9/12
11/11 [==============================] - 0s 4ms/step - loss: 0.0183 - accuracy: 0.0909
Epoch 10/12
11/11 [==============================] - 0s 5ms/step - loss: 0.0186 - accuracy: 0.0909
Epoch 11/12
11/

In [3]:
from keras.models import load_model
model = load_model('../Models/LSTM-10.keras')

ValueError: ignored

In [ ]:
# predictions with X_test data
y_LSTM = model.predict(X_test)
y_LSTM_O = scaler.inverse_transform(y_LSTM)


1/1 [==============================] - 1s 776ms/step


In [ ]:
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=pd.to_datetime(test_data.index)[7:],
    y=test_data["Close"][7:],
    mode='lines',
    name='Test Data'
))
fig.update_layout(title="Model Predictions Long-Short Term Memory")
fig.add_trace(go.Scatter(
    x=pd.to_datetime(train_data.index),
    y=train_data["Close"],
    mode='lines',
    name='Train Data'
))
fig.add_trace(go.Scatter(
    x=pd.to_datetime(test_data.index),
    y=pd.DataFrame(y_LSTM_O)[0],
    mode='lines',
    name='Prediction'
))


In [ ]:
cnt=11

In [ ]:
model.save('../Models/LSTM-'+str(cnt)+'.keras')

In [ ]:
lstmResults= pd.DataFrame({ "Date" : test_data.index[:len(y_LSTM_O)],"Close":y_LSTM_O[:,0],})
lstmResults.set_index("Date",inplace=True)
lstmResults.to_csv("lstmPrediction-"+str(cnt)+".csv")

In [ ]:

# define the parameter range
param_grid = {'neurons': [ 20, 30, 40, 50, 60],
              'optimizer': ['SGD', 'RMSprop', 'Adam'],
              'num_layers':[3,5,7,9],
              'dropout_rate': ['0.2','0.3','0.4'],
              'activation': ['relu','tanh','sigmoid'],
              'learning_rate':[0.001, 0.01]}



In [ ]:
5*3*4*3*3*2

1080

In [ ]:
1080*2/24


90.0

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=20, verbose=1)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=5)

In [ ]:
grid.fit(X_train,y_train)

ValueError: ignored